In [ ]:
import glob
import os
from natsort import natsorted
from postprocessing import *

In [ ]:
mpl.rcParams['figure.dpi']= 200

In [ ]:
simulation_directory_jet = str('/n/holyscratch01/hernquist_lab/borodina/turb_drive_center_d30/jet41_early')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

In [ ]:
i_file = 5
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory_jet + filename, "r")

In [ ]:
x = snap_data['PartType0/Jet_Tracer'][:]

In [ ]:
k1 = snap_data['Config'].attrs['BH_CONSTANT_POWER_RADIOMODE_IN_ERG_PER_S']
print(k1)

In [ ]:
plt.hist(np.log10(x[x>1e-25]), bins=100, log=True)
plt.axvline(-3, c='black', zorder=100)
plt.xlabel('jet tracer')

In [ ]:
density_arr = np.array([30, 60, 100, 200, 300])
jetpower_arr = np.array([41, 42, 43, 44])

In [ ]:
threshold_tracer = 1e-3

In [ ]:
def calculate_jet_tracer(i_file):
    result = []
    for density in density_arr:
        for jetpower in jetpower_arr:
            simulation_directory_jet = str(f'/n/holyscratch01/hernquist_lab/borodina/turb_drive_center_d{density}/jet{jetpower}_early')
            output_directory_jet = simulation_directory_jet + "/output/"
            filename = "snap_%03d.hdf5" % (i_file)
            try:
                snap_data = h5py.File(output_directory_jet + filename, "r")
                volume = snap_data['PartType0/Masses'][:]/snap_data['PartType0/Density'][:] 
                tracer = snap_data['PartType0/Jet_Tracer'][:]
                jet_volume = volume[tracer > threshold_tracer]
                jet_volume_fraction = np.sum(jet_volume)/np.sum(volume)
                result.append([density, jetpower, jet_volume_fraction])
            except:
                result.append([density, jetpower, -1])
    time = get_time_from_snap(snap_data)
    return np.array(result).T, time

In [ ]:
xaxis = np.array([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5])

for i in [0, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]:
    result, time = calculate_jet_tracer(i)
    
    fig, ax = plt.subplots(figsize=(4.5, 3))
    norm=plt.Normalize(0, 0.6)
    
    mask = (result[2] > 0)

    sc = ax.scatter(xaxis[mask], result[1][mask], c=result[2][mask], 
                    cmap='plasma', norm=norm, marker='s', s=1580, zorder=100)
    ax.tick_params(labelsize=12, direction='in')

    locs, labels = plt.xticks()  # Get the current locations and labels.
    plt.xticks([1, 2, 3, 4, 5], density_arr)  # Set text labels and properties.
    ax.set_xlim(0.5, 5.5)
    ax.set_ylim(40.5, 44.5)

    ax.set_xlabel('density [cm$^{-3}$]', fontsize=15)
    ax.set_ylabel('log (jet power)', fontsize=15)

    cbar = plt.colorbar(sc, format='%g')
    cbar.set_label(label=r'jet tracer volume fraction', size=10)

    ax.set_title("t=%.2f Myr"%(time * unit_time_in_megayr))
    plt.savefig( f'./figures/jet_tracer_{i}.png', bbox_inches='tight', dpi=300)
    plt.close()

## make gif

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = './figures/jet_tracer*.png'
ofilename = './figures/jet_tracer-jet.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=140, loop=0)